In [ ]:
#IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# DATASET

In [ ]:
path='/kaggle/input/lacaydata/train'
plt.figure(figsize=(70,70))
count=0
plant_names=[]
total_images=0
for i in os.listdir(path):
  count+=1
  plant_names.append(i)
  plt.subplot(7,7,count)

  images_path=os.listdir(path+"/"+i)
  print("Number of images of "+i+":",len(images_path),"||",end=" ")
  total_images+=len(images_path)

  image_show=plt.imread(path+"/"+i+"/"+images_path[0])

  plt.imshow(image_show)
  plt.xlabel(i)

  plt.xticks([])
  plt.yticks([])

print("Total number of images we have",total_images)

In [ ]:
print(plant_names)
print(len(plant_names))
num_classes = len(plant_names)

**IMPORTING NECESSARY LIBRARIES FOR TRAINING OF MODEL**

In [ ]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau


# RESNET50 IMPLEMENTATION USING KERAS API

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
base_model_tf=DenseNet121(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=num_classes)

In [ ]:
base_model_tf.trainable = False
pt = Input(shape=(224, 224, 3))
x = base_model_tf(pt, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)


model_main = Model(inputs=pt, outputs=x)
model_main.summary()

In [ ]:
#Image augmentation
train_datagen= ImageDataGenerator(zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,
                                                                    shear_range=0.2, horizontal_flip=True,fill_mode='nearest', validation_split=0.2)

val_datagen=ImageDataGenerator()

path_train='/kaggle/input/lacaydata/train'

path_valid='/kaggle/input/lacaydata/valid'

train= train_datagen.flow_from_directory(directory=path_train,batch_size=32,target_size=(224,224),
                                         color_mode='rgb',class_mode='categorical',seed=42,subset="training", shuffle = True)

valid=train_datagen.flow_from_directory(directory=path_train,batch_size=32,target_size=(224,224),
                                      color_mode='rgb',class_mode='categorical',subset="validation", shuffle = True)

test= val_datagen.flow_from_directory(directory=path_valid,batch_size=32,target_size=(224,224),
                                      color_mode='rgb',class_mode='categorical', shuffle = False)

In [ ]:
#CallBacks
es=EarlyStopping(monitor='val_loss',verbose=1,patience=7,mode='auto')

In [ ]:
model_main.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy', 'recall', 'precision', 'f1'])

In [ ]:
history = model_main.fit(train,validation_data=valid,epochs=20,verbose=1,callbacks=[es])

In [ ]:
model_main.save("DENSENET121_PLANT_DISEASE.h5")

In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from PIL import Image

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['loss'],color='b',label='Training loss')
plt.plot(model_main.history.history['val_loss'],color='r',label='Validation loss')
plt.xlabel("epochs")
plt.ylabel("loss_value")
plt.title("loss")


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['accuracy'],color='b',label='Training accuracy')
plt.plot(model_main.history.history['val_accuracy'],color='r',label='Validation accsuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("accuracy graph")
